# Extract SCN9A Genotype Data From UK BioBank
Author: Graeme Newton - graeme.newton@bristol.ac.uk / gwtnewton@outlook.com \
Date: 10/05/2024

This code should be run via a jupytrlab notebook on DNAnexus. \
If errors are encountered while running this script, it is likely due to insufficient RAM for annotation. For VEP, lower the --fork and or --buffer_size to fix this, or use a more powerful instance. \

Recommended instance: 2 x mem3_ssd2_v2_x4 \
Approximate runtime: 60 minutes \
Approximate cost: £0.20

In [ ]:
%%bash
# Function to download PLINK2
download_plink2() {
  wget https://s3.amazonaws.com/plink2-assets/plink2_linux_avx2_20240504.zip \
    && unzip plink2_linux_avx2_20240504.zip -d plink2_tmp \
    && mv plink2_tmp/plink2 "${INSTALL_DIR}/bin" \
    && rm -rf plink2_linux_avx2_20240504.zip plink2_tmp
}

# Function to download data using dx
download_data() {
  dx download "Bulk/Exome sequences/Population level exome OQFE variants, PLINK format - final release/ukb23158_c2_b0_v1.*"
}

# Send all outputs during installation to the log file
exec > installation_log.txt 2>&1

# Run PLINK2 download function in background
download_plink2 &

# Run data download function in background
download_data &

# Install sudo annd other in foreground due to installation order dependencies
wget https://www.sudo.ws/dist/sudo-1.9.15p5.tar.gz \
  && tar xf sudo-1.9.15p5.tar.gz \
  && cd sudo-1.9.15p5 \
  && ./configure && make && make install \
  && cd .. && rm -f sudo-1.9.15p5.tar.gz && rm -rf sudo-1.9.15p5

# Install build-essential package
sudo apt update
sudo apt install -y build-essential libmysqlclient-dev

# Install Perl modules using cpanminus
curl -L https://cpanmin.us | sudo perl - App::cpanminus
sudo cpanm Archive::Zip DBI DBD::mysql Bio::Root::Version Bio::DB::HTS

# Clone and setup VEP
git clone https://github.com/Ensembl/ensembl-vep.git
cd ensembl-vep
perl INSTALL.pl --AUTO acfp --ASSEMBLY GRCh38 --PLUGINS all --SPECIES homo_sapiens

# Download Homo sapiens cache data for VEP
curl -O https://ftp.ensembl.org/pub/release-111/variation/indexed_vep_cache/homo_sapiens_vep_111_GRCh38.tar.gz
tar xzf homo_sapiens_vep_111_GRCh38.tar.gz
rm homo_sapiens_vep_111_GRCh38.tar.gz
cd ..

In [ ]:
%%bash
plink2 --bfile ukb23158_c2_b0_v1 --chr 2 --from-bp 166185000 --to-bp 166386000 --no-categorical --geno-counts --sample-counts --recode vcf --out SCN9A_gene

In [ ]:
%%bash
# Annotate the SCN9A vcf file
./ensembl-vep/vep -i ./SCN9A_gene.vcf --fork 4 --cache --dir_cache ./ensembl-vep --species homo_sapiens --pick --everything --canonical --vcf --buffer_size 100 --force_overwrite -o SCN9A_gene_anno.vcf

In [ ]:
%%bash
# Compress the genotype files
gzip SCN9A_gene*.vcf

# Upload alloutput files to your DNAnexus project
dx upload SCN9A_gene*